In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib as mat
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.pyplot import figure
from sklearn import preprocessing

In [2]:
cd /Users/mihaelanemes/Desktop/s2ds/data/Data/

/Users/mihaelanemes/Desktop/s2ds/data/Data


In [3]:
# read csv file - note all data is for Wales only

food_and_you_wales = pd.read_csv("food_and_you_all_waves_wales.csv")

# remove values that are 'refused to answer', 'unknown' etc as nan

food_and_you_wales = food_and_you_wales.replace ([-9, -8, -1, 98], np.nan)   

# replace 'not applicable' in specific columns

cols = ['q4_1_4', 'q4_1_5a', 'Q4_1_5_comb', 'q4_1_6', 'q4_1_7', 'q4_1_8a', 'q4_1_8b', 'sanspray', 'q4_1_11',
        'q4_1_12', 'q4_1_13', 'q4_1_14', 'q4_1_15', 'q4_1_16', 'q4_1_17', 'q4_1_18', 'q4_1_19']

food_and_you_wales[cols] = food_and_you_wales[cols].replace([5], np.nan)

# standardize values by column - not needed in this script

food_and_you_wales_std = preprocessing.StandardScaler(food_and_you_wales, with_mean=False)

food_and_you_wales.count()

SerialNo              1757
RespSex               1757
age_dv                1029
bhhsize2               540
below6                1027
below16               1755
marstat2              1754
hhdinc                1332
workstat2              540
region_dv             1757
religion_dv           1748
wimd_2014_quintile    1029
nimd_2010_quintile       0
nimd2017_quintile        0
UrbanRuralInd         1757
surveyyear            1757
country_dv            1757
psu_dv                1757
stratum_all_dv        1757
combinedW1_5weight    1757
countryW1_5weight     1757
CkRes                 1029
resfdprp              1029
CkFreq                1029
bcook3                1029
dietary               1757
fdreac_dv             1029
reacalle               196
reacint                196
reaccoe                196
                      ... 
HeardFSA               540
C1                     509
C2                     517
C3                     507
C4                     517
C5                     508
C

In [4]:
# select data of interest for a particular plot

temp = food_and_you_wales[['surveyyear', 'age_dv', 'RespSex', 'religion_dv','hhdinc', 'workstat2', 
                           'wimd_2014_quintile',
                           'Q4_1_5_comb', 'q4_1_6', 'q4_1_7', 'q4_1_8a', 'q4_1_8b']]  
print(temp)


      surveyyear  age_dv  RespSex  religion_dv  hhdinc  workstat2  \
0              1     NaN        2          1.0     3.0        NaN   
1              1     NaN        2          1.0     NaN        NaN   
2              1     NaN        2          1.0     4.0        NaN   
3              1     NaN        1          1.0     4.0        NaN   
4              1     NaN        2          1.0     4.0        NaN   
5              1     NaN        1          3.0     2.0        NaN   
6              1     NaN        2          1.0     2.0        NaN   
7              1     NaN        2          1.0     NaN        NaN   
8              1     NaN        2          1.0     1.0        NaN   
9              1     NaN        1          1.0     NaN        NaN   
10             1     NaN        2          1.0     NaN        NaN   
11             1     NaN        2          3.0     3.0        NaN   
12             1     NaN        2          1.0     4.0        NaN   
13             1     NaN        2 

In [13]:
# do triple groupby for each var of interest: wave, answer to the Q and demographic 

gender_Q4_1_5_comb = temp.groupby(['RespSex', 'surveyyear', 'Q4_1_5_comb']).agg({'Q4_1_5_comb':'sum'})
gender_Q4_1_5_comb = gender_Q4_1_5_comb.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

age_Q4_1_5_comb = temp.groupby(['age_dv', 'surveyyear', 'Q4_1_5_comb']).agg({'Q4_1_5_comb':'sum'})
age_Q4_1_5_comb = age_Q4_1_5_comb.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

religion_Q4_1_5_comb = temp.groupby(['religion_dv', 'surveyyear', 'Q4_1_5_comb']).agg({'Q4_1_5_comb':'sum'})
religion_Q4_1_5_comb = religion_Q4_1_5_comb.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

income_Q4_1_5_comb = temp.groupby(['hhdinc', 'surveyyear', 'Q4_1_5_comb']).agg({'Q4_1_5_comb':'sum'})
income_Q4_1_5_comb = income_Q4_1_5_comb.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

poverty_Q4_1_5_comb = temp.groupby(['wimd_2014_quintile', 'surveyyear', 'Q4_1_5_comb']).agg({'Q4_1_5_comb':'sum'})
poverty_Q4_1_5_comb = poverty_Q4_1_5_comb.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

grid_data = pd.DataFrame (['gender_Q4_1_5_comb', 'age_Q4_1_5_comb', 'religion_Q4_1_5_comb',
                         'income_Q4_1_5_comb', 'poverty_Q4_1_5_comb']) 

# reset index to keep the other variables not only the Q of interest


In [15]:
# custom function for plotting to specify the % number on top

def f(x,y, **kwargs):
    ax = sns.lineplot(x,y,**kwargs)
    for i in range(len(x)):
        ax.annotate(str(y.values[i]), xy=(x.values[i], y.values[i]),fontsize=8,
                    xytext = (0,10), textcoords="offset points",
                color=kwargs.get("color","k"), 
                va='center', ha='center')
        

In [16]:
# plot features of interest from the created 'temp' subset

# sns.set(style="ticks")
# grid = sns.FacetGrid(gender_Q4_1_5_comb, col="surveyyear", hue="Q4_1_5_comb", palette="tab20c")

g = sns.FacetGrid(gender_Q4_1_5_comb, col='RespSex', hue='Q4_1_5_comb') 
g = (g.map(f, 'surveyyear', 'Q4_1_5_comb')
     .add_legend())
g.set_titles(pad = 20) # '{row_name}' ' | ' '{col_name}'
plt.subplots_adjust(hspace=0.4, wspace=0.4)
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True, labelleft=True)

KeyError: 'RespSex'

In [15]:
# clunky code - don't run!


# save responses from each wave in a new matrix variable

wave1 = [1]
food_and_you_wales_wave1 = food_and_you_wales.loc [food_and_you_wales['surveyyear'].isin(wave1)]

wave2 = [2]
food_and_you_wales_wave2 = food_and_you_wales.loc [food_and_you_wales['surveyyear'].isin(wave2)]

wave3 = [3]
food_and_you_wales_wave3 = food_and_you_wales.loc [food_and_you_wales['surveyyear'].isin(wave3)]

wave4 = [4]
food_and_you_wales_wave4 = food_and_you_wales.loc [food_and_you_wales['surveyyear'].isin(wave4)]

wave5 = [5]
food_and_you_wales_wave5 = food_and_you_wales.loc [food_and_you_wales['surveyyear'].isin(wave5)]

# create var to save all waves in one place

all_waves = { 'wave1':['food_and_you_wales_wave1'], 'wave2':['food_and_you_wales_wave2'],
             'wave3':['food_and_you_wales_wave3'], 'wave4':['food_and_you_wales_wave4'], 
             'wave5':['food_and_you_wales_wave5'] }

# calculate % of each response 

var_wave1 = food_and_you_wales_wave1['Q4_1_5_comb'].value_counts()/food_and_you_wales_wave1['Q4_1_5_comb'].count()*100
var_wave2 = food_and_you_wales_wave2['Q4_1_5_comb'].value_counts()/food_and_you_wales_wave2['Q4_1_5_comb'].count()*100
var_wave3 = food_and_you_wales_wave3['Q4_1_5_comb'].value_counts()/food_and_you_wales_wave3['Q4_1_5_comb'].count()*100
var_wave4 = food_and_you_wales_wave4['Q4_1_5_comb'].value_counts()/food_and_you_wales_wave4['Q4_1_5_comb'].count()*100
var_wave5 = food_and_you_wales_wave5['Q4_1_5_comb'].value_counts()/food_and_you_wales_wave5['Q4_1_5_comb'].count()*100

x_axis = [0,1,2,3,4]
y_axis = [var_wave1,var_wave2, var_wave3, var_wave4, var_wave5]


# plot by % - show trend over waves; line plot with percentage for data point and number of responses in brackets 
# time-series line plot - x-axis: waves; y-axis: %; data point labels: % value and no of responses; 
# example using data from fig 2.7: Q4_1_5_comb; q4_1_6; q4_1_7; q4_1_8a; q4_1_8b

%matplotlib inline
# plt.style.use('seaborn-whitegrid'
fig_raw_meat = plt.plot (x_axis, y_axis, 'o', linewidth=1.0, linestyle="-")

plt.xticks([0, 1, 2, 3, 4],
          ['wave_1','wave_2','wave_3','wave_4','wave_5'])
plt.ylabel('percentage responses')

plt.legend(['Never', 'Sometimes', 'Most of the time', 'Always'], loc='center left', bbox_to_anchor=(1, 0.5))

plt.title('how often do you wash raw meat excl chicken')

     SerialNo  RespSex  age_dv  bhhsize2  below6  below16  marstat2  hhdinc  \
0      100027        2     NaN       NaN     NaN      1.0       1.0     3.0   
1      100037        2     NaN       NaN     NaN      2.0       1.0     NaN   
2      100129        2     NaN       NaN     NaN      2.0       1.0     4.0   
3      100130        1     NaN       NaN     NaN      2.0       1.0     4.0   
4      100285        2     NaN       NaN     NaN      2.0       1.0     4.0   
5      100314        1     NaN       NaN     NaN      1.0       1.0     2.0   
6      100406        2     NaN       NaN     NaN      2.0       2.0     2.0   
7      100591        2     NaN       NaN     NaN      2.0       1.0     NaN   
8      100815        2     NaN       NaN     NaN      2.0       2.0     1.0   
9      100969        1     NaN       NaN     NaN      2.0       2.0     NaN   
10     101144        2     NaN       NaN     NaN      1.0       1.0     NaN   
11     101169        2     NaN       NaN     NaN    